In [ ]:
%reset -f
import datetime
import os.path
import sys
import pandas as pd
import talib
import numpy as np
import talib as ta
from ta_calculat import runmain
import threading
import pickle
import time
import gc

print(time.ctime())
class backtest():

    def __init__(self,datasignal=None,leverage=1,BUY=True,SELL=True,total_size=False,
                 trigger=None,Commission=None,sl=None,tp=None,value=None,stoptraill=None,size=1):
        self.order=[('index','type position')]
        self.leverage=leverage
        self.leveragevalue=value*self.leverage-value
        self.datasignal=datasignal
        #cash is all money without open orders
        self.controlorder=['sl_tp','stoptraill']
        self.trigger=trigger    # sl_tp  or  stoptraill
        self.size=size
        self.total_size=total_size
        self.gross=None
        self.profit=[('index','traillist','size','start-price','stopprice','open','high','low','percent profit (%)','gain','type position')]
        self.lose=[('index','traillist','size','start-price','stopprice','open','high','low','percent lose (%)','lose','type position')]
        self.cash=value*self.leverage
        self.actionprice=None
        self.value=value*self.leverage
        self.sl=sl
        self.tp=tp
        self.stoptraill=stoptraill
        self.pricestoptraill=None
        self.Commission=Commission
        self.typeposition=None
        self.buyorsell=pd.DataFrame()
        self.BUY=BUY
        self.SELL=SELL
        self.mainindex=None
        self.iterindex=None
        self.main_data=None
        self.iter_data=None

    def sizer(self,n):
        try:
            if self.total_size==True:
                sizer=self.cash/self.main_data.loc[n,'open']
            else:
                sizer=self.size
                
        except Exception as error:
            print(error)
            print('***',n,'****')
        return sizer

    def getdata(self):
        global main_timeframe
        global iter_timeframe
        dff = pd.read_pickle("data_pairs_kucoin_future_%s.pkl" % (iter_timeframe)).loc[0,'pair']
        dff['date'] = dff['date'].astype(str) +' '+ dff['time'].astype(str)
        dff['date']= pd.to_datetime(dff['date'], format='%d/%m/%Y %H:%M:%S')
        dff.set_index('date', inplace=True)
        dff.drop('time', axis=1,inplace=True)
        self.iterindex=dff.index
        dff.reset_index(drop=True, inplace=True)

        df = pd.read_pickle("data_pairs_kucoin_future_%s.pkl" % (main_timeframe)).loc[0,'pair']
        df['date'] = df['date'].astype(str) +' '+ df['time'].astype(str)
        df['date']= pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M:%S')
        df.set_index('date', inplace=True)
        df.drop('time', axis=1,inplace=True)
        self.mainindex=df.index
        df.reset_index(drop=True, inplace=True)
        return df , dff

    def buy(self,j):
        j=self.iterindex.get_loc(j)
        starti=j
#         print('\n',j,'buy','\n')
        self.typeposition='buy'
        self.order.append((j,self.typeposition))
        self.actionprice=self.iter_data.loc[j,'open']
        self.pricestoptraill=self.actionprice
        self.cash=self.value - self.actionprice*self.size
        traillist=[]
        b=0
        while b==0:
#             print(j,end='-')

            if self.trigger==self.controlorder[0]:

                if self.iter_data.loc[i,'low']<=(1-self.sl)*self.actionprice or self.iter_data.loc[i,'high']>=(1+self.tp)*self.actionprice or j>=len(self.iter_data)-3:
                    self.gross=((self.iter_data.loc[i,'close']-self.actionprice)/self.actionprice)*100

                    comis=self.iter_data.loc[i,'close']*self.size*self.Commission
                    gain=(self.iter_data.loc[i,'close']*self.size-self.actionprice*self.size)-comis
                    
                    

                    self.value=self.value+gain

                    self.cash=self.cash+self.iter_data.loc[i,'close']*self.size-comis
                    

                    if self.gross>=0:
                        self.profit.append(('start:%s - end:%s' % (starti,i),self.iter_data.loc[starti,'open'],self.iter_data.loc[i,'high'],
                                            self.iter_data.loc[i,'low'],(1-self.stoptraill)*self.iter_data.loc[i,'low'],
                                            (self.stoptraill+1)*self.iter_data.loc[i,'high'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start:%s - end:%s' % (starti,i),self.iter_data.loc[starti,'open'],self.iter_data.loc[i,'high'],
                                            self.iter_data.loc[i,'low'],(1-self.stoptraill)*self.iter_data.loc[i,'low'],
                                            (self.stoptraill+1)*self.iter_data.loc[i,'high'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    #profit
                    break

            elif self.trigger==self.controlorder[1]:
                
                if self.iter_data.loc[j,'low'] - self.Commission*self.iter_data.loc[j,'low']<=\
                (1-(self.sl/self.leverage))*self.pricestoptraill\
                or self.iter_data.loc[j,'close'] - self.Commission*self.iter_data.loc[j,'close']<=\
                (1-(self.sl/self.leverage))*self.pricestoptraill\
                or j>=len(self.iter_data)-3:
                    
                    if  self.iter_data.loc[j,'low'] - self.Commission*self.iter_data.loc[j,'low']<=\
                    (1-(self.sl/self.leverage))*self.pricestoptraill:
                        stopprice=self.iter_data.loc[j,'low']
                    elif self.iter_data.loc[j,'close'] - self.Commission*self.iter_data.loc[j,'close']<=\
                    (1-(self.sl/self.leverage))*self.pricestoptraill:
                        stopprice=self.iter_data.loc[j,'close']
                    else:
                        stopprice=self.iter_data.loc[j,'close']
                    #-----------------------------------------------------------    
                    self.gross=((stopprice-self.actionprice)/self.actionprice)*100

                    comis=stopprice*self.size*self.Commission
                    gain=(stopprice*self.size - self.actionprice*self.size) - comis

                    self.value=self.value + gain

                    self.cash=self.cash + stopprice*self.size - comis
                    #----------------------------------------------------------- 
                    if self.gross>=0:
                        self.profit.append(('start:%s - end:%s' % (starti,j),traillist,
                                            self.size,
                                            self.actionprice,stopprice,
                                            self.iter_data.loc[j,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start:%s - end:%s' % (starti,j),traillist,
                                          self.size,
                                          self.actionprice,stopprice,
                                          self.iter_data.loc[j,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    #profit
                    break
                elif (self.iter_data.loc[j,'high']-self.iter_data.loc[j,'high']*self.Commission)/self.pricestoptraill>=\
                (1+(self.stoptraill/self.leverage))\
                or (self.iter_data.loc[j,'close']-self.iter_data.loc[j,'close']*self.Commission)/self.pricestoptraill>=\
                (1+(self.stoptraill/self.leverage)):
                    
                    if  (self.iter_data.loc[j,'high']-self.iter_data.loc[j,'high']*self.Commission)/self.pricestoptraill>=\
                (1+(self.stoptraill/self.leverage)):
                        
                        self.pricestoptraill=self.iter_data.loc[j,'high']
                        
                    elif (self.iter_data.loc[j,'close']-self.iter_data.loc[j,'close']*self.Commission)/self.pricestoptraill>=\
                (1+(self.stoptraill/self.leverage)):
                        
                        self.pricestoptraill=self.iter_data.loc[j,'close']
                    
                    traillist.append(self.pricestoptraill)

            j+=1

        return self.iterindex[j]

    def sell(self,j):
        j=self.iterindex.get_loc(j)
        starti=j
#         print('\n',j,'sell','\n')
        self.typeposition='sell'
        self.order.append((j,self.typeposition))
        self.actionprice=self.iter_data.loc[j,'open']
        self.pricestoptraill=self.actionprice
        self.cash=self.value-self.actionprice*self.size
        traillist=[]
        b=1

        while b==1:
#             print(j,end='-')

            if self.trigger==self.controlorder[0]:
                if self.iter_data.loc[j,'high']>=(1+self.sl)*self.actionprice \
                or self.iter_data.loc[j,'low']<=(1-self.tp)*self.actionprice or j>=len(self.iter_data)-3:
                    self.gross=((self.actionprice-self.iter_data.loc[j,'close'])/self.actionprice)*100

                    comis=((self.actionprice*self.size-self.iter_data.loc[j,'close']*self.size)+\
                    self.actionprice*self.size)*self.Commission
                    
                    self.cash=self.cash+((self.actionprice*self.size-self.iter_data.loc[j,'close']*self.size)+\
                    self.actionprice*self.size)-comis

                    gain=(self.actionprice*self.size-self.iter_data.loc[j,'close']*self.size)-comis

                    self.value=self.value+gain

                    if self.gross>=0:
                        self.profit.append(('start:%s - end:%s' % (starti,j),self.iter_data.loc[starti,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],(1-self.stoptraill)*self.iter_data.loc[j,'low'],
                                            (self.stoptraill+1)*self.iter_data.loc[j,'high'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                        
                    if self.gross<0:
                        self.lose.append(('start:%s - end:%s' % (starti,j),self.iter_data.loc[starti,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],(1-self.stoptraill)*self.iter_data.loc[j,'low'],
                                            (self.stoptraill+1)*self.iter_data.loc[j,'high'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    break

            elif self.trigger==self.controlorder[1]:

                if (self.iter_data.loc[j,'high'] + self.Commission*self.iter_data.loc[j,'high'])>=\
                (1+(self.sl/self.leverage))*self.pricestoptraill \
                or (self.iter_data.loc[j,'close'] + self.Commission*self.iter_data.loc[j,'close'])>=\
                (1+(self.sl/self.leverage))*self.pricestoptraill\
                or j>=len(self.iter_data)-3:
                    
                    if  self.iter_data.loc[j,'high'] + self.Commission*self.iter_data.loc[j,'high']>=\
                    (1+(self.sl/self.leverage))*self.pricestoptraill:
                        stopprice=self.iter_data.loc[j,'high']
                    elif self.iter_data.loc[j,'close'] + self.Commission*self.iter_data.loc[j,'close']>=\
                    (1+(self.sl/self.leverage))*self.pricestoptraill:
                        stopprice=self.iter_data.loc[j,'close']
                    else:
                        stopprice=self.iter_data.loc[j,'close']
                    #-----------------------------------------------------------     
                    self.gross=((self.actionprice-stopprice)/self.actionprice)*100
                    
                    comis=((self.actionprice*self.size-stopprice*self.size)+\
                    self.actionprice*self.size)*self.Commission


                    gain=(self.actionprice*self.size-stopprice*self.size)-comis

                    self.value=self.value+gain
                    
                    self.cash=self.cash+((self.actionprice*self.size-stopprice*self.size)+\
                    self.actionprice*self.size)-comis
                    #-----------------------------------------------------------  

                    if self.gross>=0:
                        self.profit.append(('start:%s - end:%s' % (starti,j),traillist,
                                            self.size,
                                            self.actionprice,stopprice,
                                          self.iter_data.loc[j,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start:%s - end:%s' % (starti,j),traillist,
                                          self.size,
                                          self.actionprice,stopprice,
                                          self.iter_data.loc[j,'open'],self.iter_data.loc[j,'high'],
                                            self.iter_data.loc[j,'low'],
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    break
                elif (self.iter_data.loc[j,'low']+self.iter_data.loc[j,'low']*self.Commission)/self.pricestoptraill<=\
                (1-(self.stoptraill/self.leverage))\
                or (self.iter_data.loc[j,'close']+self.iter_data.loc[j,'close']*self.Commission)/self.pricestoptraill<=\
                (1-(self.stoptraill/self.leverage)):
                    
                    if  (self.iter_data.loc[j,'low']+self.iter_data.loc[j,'low']*self.Commission)/self.pricestoptraill<=\
                (1-(self.stoptraill/self.leverage)):
                        
                        self.pricestoptraill=self.iter_data.loc[j,'low']
                        
                    elif (self.iter_data.loc[j,'close']+self.iter_data.loc[j,'close']*self.Commission)/self.pricestoptraill<=\
                (1-(self.stoptraill/self.leverage)):
                        
                        self.pricestoptraill=self.iter_data.loc[j,'close']
                    
                    traillist.append(self.pricestoptraill)

            j+=1

        return self.iterindex[j]

    def run(self):
        ilist=[]
        self.main_data,self.iter_data=self.getdata()
        c=0
        x=0
        try:
            while x==0:
    #                 print(c,end='-')
                ilist.append(c)
                if c>= len(self.main_data)-5:
                    break

                if self.BUY==True:
                    if self.datasignal.loc[c,'sellorbuy']==1: # 1==buy
                        self.size=self.sizer(c+1)
                        if int(self.main_data.loc[c+1,'open']*self.size)<=int(self.cash):
                            c=self.buy(self.mainindex[c+1])
                            if c in self.mainindex:
                                c=self.mainindex.get_loc(c)
                            else:
                                b=0
                                t=self.iterindex.get_loc(c)
                                t+=1
                                while b==0:
                                    if t>len(self.iterindex)-1:
                                        c=len(self.mainindex)-2
                                        break
                                    c=self.iterindex[t]
                                    if c in self.mainindex:
                                        c=self.mainindex.get_loc(c)
                                        break
                                    t+=1
                        else:

                            print('********************** no enough cash for buy **********************')
                            print(c,self.size,self.main_data.loc[c+1,'open']*self.size,'<=',self.cash)
                            continue


                if self.SELL==True:

                    if self.datasignal.loc[c,'sellorbuy']==0: # 0==sell
                        self.size=self.sizer(c+1)
                        if int(self.main_data.loc[c+1,'open']*self.size)<=int(self.cash):
                            c=self.sell(self.mainindex[c+1])
                            if c in self.mainindex:
                                c=self.mainindex.get_loc(c)
                            else:
                                b=0
                                t=self.iterindex.get_loc(c)
                                t+=1
                                while b==0:
                                    if t>len(self.iterindex)-1:
                                        c=len(self.mainindex)-2
                                        break
                                    c=self.iterindex[t]
                                    if c in self.mainindex:
                                        c=self.mainindex.get_loc(c)
                                        break

                                    t+=1
                        else:
                            print('**********************no enough cash for sell **********************')
                            print(c,self.size,self.main_data.loc[c+1,'open']*self.size,'<=',self.cash)
                            continue
                c+=1
        except Exception as error:
            print('#',c,'#')
            print(error)


        self.order=pd.DataFrame(self.order)
        self.order.columns = self.order.iloc[0] #set the header row as the df header
        self.order = self.order[1:] #take the data less the header row
        self.profit=pd.DataFrame(self.profit)
        self.profit.columns = self.profit.iloc[0]
        self.profit = self.profit[1:]
        self.lose=pd.DataFrame(self.lose)
        self.lose.columns = self.lose.iloc[0]
        self.lose = self.lose[1:]
        
        if self.leverage>1:
            self.value=self.value-self.leveragevalue
  
        return self.order ,self.profit ,self.lose ,self.value,ilist

    
    
def startbacktest(s,l,st,inde,column):
    global alldata
    global queue
    result_order=pd.DataFrame(index=inde,columns=column)
    result_profit=pd.DataFrame(index=inde,columns=column)
    result_lose=pd.DataFrame(index=inde,columns=column)
    result_value=pd.DataFrame(index=inde,columns=column)
    list_i=pd.DataFrame(index=inde,columns=column)
    for j in sellorbuy.columns:
        for i in sellorbuy.index:
            if type(sellorbuy.loc[i,j])==int:
                continue
#             print('\n',i , j,s,l,st,'------------------------------------------------------------','\n' )
            datasignal=sellorbuy.loc[i,j].copy()
            bt=backtest(datasignal=datasignal,total_size=True,value=100,trigger='stoptraill',
                        Commission=0.002,stoptraill=st,sl=s,leverage=l)
            order,profit,lose,value,ilist=bt.run()
            result_order.at[i,j]=order
            result_profit.at[i,j]=profit
            result_lose.at[i,j]=lose
            result_value.at[i,j]=value
            list_i.at[i,j]=ilist

    ui=1
    while ui==1:
        if queue==0:
            queue=1
            break
        time.sleep(1)
    alldata.at['result_order','%s-%s-%s' % (s,l,st)]=result_order
    alldata.at['result_profit','%s-%s-%s' % (s,l,st)]=result_profit
    alldata.at['result_lose','%s-%s-%s' % (s,l,st)]=result_lose
    alldata.at['result_value','%s-%s-%s' % (s,l,st)]=result_value
    alldata.at['listi','%s-%s-%s' % (s,l,st)]=list_i
    queue=0
    return
queue=0

sl=list(np.around(np.array(np.arange(0.01, 0.06, 0.01).tolist()),2))
stoptraill=list(np.around(np.array(np.arange(0.01, 0.25, 0.01).tolist()),2))
leverage=list(np.around(np.array(np.arange(1, 16, 1).tolist()),2))

main_timeframe='5m'
iter_timeframe='1m'
iter_limit=2700
main_limit=int(iter_limit/5)
# sellorbuy=runmain(main_timeframe,main_limit)
# sellorbuy.to_pickle("sellorbuy_main.pkl")
# time.sleep(5)
# runmain(iter_timeframe,iter_limit)

sellorbuy = pd.read_pickle("sellorbuy_main.pkl")
inde=sellorbuy.index
column=sellorbuy.columns
#-----------------------------------------------------
colmalldata=[]
for s in sl:
    for l in leverage:
        for st in stoptraill:
            colmalldata.append('%s-%s-%s' % (s,l,st))
alldata=pd.DataFrame(index=['result_order','result_profit','result_lose','result_value','listi'],columns=colmalldata)

for s in sl:
    print('\n',s,end='-\n')
    for l in leverage:
        print('\n',l,end='-\n')
        threads=[]
        for st in stoptraill:
            print(st,end='-')
#             startbacktest(s,l,st,inde,column,)

            thread=threading.Thread(target=startbacktest,args=(s,l,st,inde,column,))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
alldata.to_pickle("alldata.pkl")
#--------------------------------------------
print('\n',time.ctime())
#-----------------------------------------------------
# result_order=pd.DataFrame(index=inde,columns=column)
# result_profit=pd.DataFrame(index=inde,columns=column)
# result_lose=pd.DataFrame(index=inde,columns=column)
# result_value=pd.DataFrame(index=inde,columns=column)
# list_i=pd.DataFrame(index=inde,columns=column)
# for j in sellorbuy.columns:
#     for i in sellorbuy.index:
#         if type(sellorbuy.loc[i,j])==int:
#             continue
#         print('\n',i , j,'------------------------------------------------------------','\n' )
#         datasignal=sellorbuy.loc[i,j].copy()
#         bt=backtest(datasignal=datasignal,total_size=True,value=100,trigger='stoptraill',
#                     Commission=0.002,stoptraill=0.05,sl=0.05,leverage=1)
#         order,profit,lose,value,ilist=bt.run()
#         result_order.at[i,j]=order
#         result_profit.at[i,j]=profit
#         result_lose.at[i,j]=lose
#         result_value.at[i,j]=value
#         list_i.at[i,j]=ilist


In [ ]:
display(result_profit.loc['macd','signal1'])
display(result_lose.loc['macd','signal1'])
display(result_order.loc['macd','signal1'])

In [ ]:
import pandas as pd
df = pd.read_pickle("data_pairs_kucoin_future_1m.pkl").loc[0,'pair']
df['date'] = df['date'].astype(str) +' '+ df['time'].astype(str)
df['date']= pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M:%S')
df.set_index('date', inplace=True)
df.drop('time', axis=1,inplace=True)
df